# What is mlvtrans?

mlvtrans is a python package that implements the algorithm explained in Appendix C of the paper [Logical Clifford gates with multilevel transversality](https://scholar.google.co.jp/citations?user=hIPtQG8AAAAJ&hl=ja). For any self-dual quantum CSS code which satisfies certain conditions (see Theorem 1. in [Logical Clifford gates with multilevel transversality](https://scholar.google.co.jp/citations?user=hIPtQG8AAAAJ&hl=ja)), mlvtrans constructs a compatible symplectic basis, which is a symplectic basis such that (1) transversal logical Hadamard gates $\bigotimes_{j=1}^{k} \bar{H}_j$ can be implemented by transversal physical Hadamard gates $\bigotimes_{i=1}^{n} H_i$, and (2) for any $(a_1,\dots,a_k)\in\{-1,1\}^k$, transversal logical phase-type gates $\bigotimes_{j=1}^{k} \bar{S}_j^{a_j}$ can be implemented by transversal physical phase-type gates $\bigotimes_{i=1}^{n} S_i^{b_i}$ for some $(b_1,\dots,b_n)\in\{-1,1\}^n$. mlvtrans also outputs such $(b_1,\dots,b_n)\in\{-1,1\}^n$ for any choice of $(a_1,\dots,a_k)\in\{-1,1\}^k$.

# Install the python package

You can install the package mlvtrans by running `pip install mlvtrans`, and then import it with `import mlvtrans`.

In [ ]:
pip install mlvtrans

In [1]:
import mlvtrans
print(mlvtrans.__version__)

0.1.1


# How to use mlvtrans

You can execute all the procedures that mlvtrans offers by running `mlvtrans.run()`:

- `mlvtrans.run(H_matrix, A_matrix, desired_logical_S)`
    <br>It performs all the following steps: execute Algorithm 1 to obtain an initial sympletic basis, apply Lemma 2 to obtain a compatible symplectic basis, determine the appropriate transversal physical phase-type gates, and finally conduct a test to ensure that the obtained sympletic basis is compatible, and the obtained physical phase-type gates correspond to the desired logical phase-type gates.

    **Args:**
        <br>H_matrix: A parity check matrix expressed by `np.array` (each element is 0 or 1).
        <br>A_matrix: A set of coset representatives expressed by `np.array` (each element is 0 or 1).
        <br>desired_logical_S: Desired transversal logical phase-type gates provided in tuple format (each element is 1 or -1).


- `mlvtrans.run_auto_coset(H_matrix, desired_logical_S)`
<br>With this functions, you only need to provide a parity check matrix and desired logical phase-type gates. Coset representatives are automatically calculated using Stim’s `stim.Tableau.from_stabilizers`. You have to install Stim to use this function. This functio is especially useful when only the stabilizers of a self-dual CSS code are known, and you do not know the coset representatives, or logical operators.

There are several built-in examples of self-dual CSS codes:
- `mlvtrans.examples(example_choice)`
    - example_choice="15_7_3"
        <br> [[15,7,3]] quantum Hamming code
    - example_choice="31_11_5"
        <br> [[31,11,5]] quantum BCH code
    - example_choice="16_4_3"
        <br> [[16,4,3]] color code
    - example_choice="6_2_2"
        <br> [[6,2,2]] code
    - example_choice="4_2_2"
        <br> [[4,2,2]] code
    - example_choice="19_1_5"
        <br> [[19,1,5]] distance-5 hexagonal color code


Although it is enough to just run `mlvtrans.run()` to execute the whole procedures, there are also functions available to execute each part separately:
- `mlvtrans.alg1(A_matrix)`
    <br>Execute Algorithm 1.

    **Args:**
    <br>A_matrix: A set of coset replesentatives expressed by `np.array`.

    **Returns:**
    <br>Two lists (for logical $X$ and $Z$) corresponding to a sympletic basis obtained by executing Algorithm 1.

- `mlvtrans.lemma2(Lx, Lz)`
    <br>Execute Lemma 2.

    **Args:**
    <br> Lx, Lz: The lists corresponding to a a sympletic basis.

    **Returns:**
    <br> Two lists (for logical $X$ and $Z$) corresponding to a sympletic basis obtained by applying Lemma 2.
- `mlvtrans.phys_S_for_logi_S(H_matrix, desired_logical_S, Lx)`
    <br>Find the transversal physical phase-type gates whose logical actions are the desired ones.

    **Args:**
    <br> H_matrix: A parity check matrix expressed by `np.array`.
    <br> desired_logical_S: Desired logical phase-type gates provided in tuple format.
    <br> Lx: The list of an $X$ part (can be $Z$ part) of a compatible sympletic basis.

    **Returns:**
     <br> A configuration of transversal physical phase-type gates represented by `np.ndarray`.
- `mlvtrans.check_compatible(H_matrix, Lx, Lz)`
    <br>Check whether the input sympletic basis is compatible. This function verifies whether each pair of logical operators shares the same odd-weight support, whether every logical operator commutes with the stabilizer group, and whether the logical operators have the appropriate commutation relations among themselves.

    **Args:**
    <br> H_matrix: A parity check matrix expressed by `np.array`.
    <br> Lx: The list of an $X$ part of a sympletic basis.
    <br> Lz: The list of a $Z$ part of a sympletic basis.

    **Returns:**
     <br>  True if the input sympletic basis is compatible; False otherwise.
- `mlvtrans.check_S(H_matrix, desired_logical_S, Lx, physical_S)`
    <br>Check whether the input physical phase-type gates are logical gates (i.e., they preserve the stabilizer group) and check whether the resulting logical gates are the same as what we want.

    **Args:**
    <br> H_matrix: A parity check matrix expressed by `np.array`.
    <br> desired_logical_S: Desired logical phase-type gates provided in tuple format.
    <br> Lx: The list of an $X$ part (can be $Z$ part) of a compatible sympletic basis.
    <br> physical_S: The output of `mlvtrans.phys_S_for_logi_S()`.
  
    **Returns:**
    <br>  True if the logical actions match the ones specified by `desired_logical_S`; False otherwise.

# Demonstrations 

Here, we show some examples to construct a compatible symplectic basis and transversal phase-type gates for self-dual CSS codes.

## 1. &nbsp; [[15,7,3]] quantum Hamming code 

Define a parity check matrix `H` and a set of coset representatives `A` using `np.array`, and define a desired choice `S` of logical phase-type gates (the combination of logical phase-type gates for which you want to find the corresponding combination of physical phase-type gates) using a tuple, starting from the 0th logical qubit (in 0-based indexing), where $\bar{S}$ is represented by 1 and $\bar{S}^\dagger$ by –1. Note that a set of coset representatives `A` corresponds to one type ($X$ or $Z$) of logical operators.

In [2]:
import numpy as np
H = np.array([
    [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
    [0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1],
    [0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
], dtype=int)
A = np.array([
    [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
    [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1],
    [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
], dtype=int)
S = (1, 1, 1, 1, 1, 1, 1)

Then, `mlvtrans.run()` gives a compatible sympletic basis and transversal physical phase-type gates that implement the transversal logical phase-type gates specified by `S`:

In [4]:
mlvtrans.run(H_matrix=H, A_matrix=A, desired_logical_S=S)

Parity-check matrix H:
[[1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
 [0 1 1 0 0 1 1 0 0 1 1 0 0 1 1]
 [0 0 0 1 1 1 1 0 0 0 0 1 1 1 1]
 [0 0 0 0 0 0 0 1 1 1 1 1 1 1 1]]

Coset representatives matrix A:
[[0 0 1 0 0 0 1 0 0 0 1 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1]
 [0 0 0 0 1 0 1 0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 0 0 0 1 1 0 0 1 1]
 [0 0 0 0 0 0 0 0 1 0 1 0 1 0 1]
 [0 0 0 0 0 1 1 0 0 0 0 0 0 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]

Desired logical phase-type gates:
 0  1  2  3  4  5  6 
 S̅  S̅  S̅  S̅  S̅  S̅  S̅ 
Indices with S̅†: 

Initial symplectic basis from Algorithm 1:
Pair 1:
  Lx = [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]   support = (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14)   weight = 15
  Lz = [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]   support = (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14)   weight = 15
Pair 2:
  Lx = [0 0 1 0 0 0 1 0 0 0 1 0 0 0 1]   support = (2, 6, 10, 14)   weight = 4
  Lz = [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1]   support = (11, 12, 13, 14)   weight = 4
Pair 3:
  Lx = [0 0 0 

## 2. &nbsp; distance-5 hexagonal color code

The hexagonal color code is a clear example of a code that, despite having stabilizers that are not doubly-even, can implement logical phase-type gates via transversal physical phase-type gates. Here, we input the code by specifying indices, using a function `convert_index_to_matrix(num_qubit, num_logical_qubits, h_indices, a_indices, logical_S_dict)`. With this function, you can define a parity check matrix and a set of coset representatives by specifying the indices corresponding to the 1 entries in them, as well as the indices of the logical qubits on which $\bar{S}^\dagger$ gates are intended to act. This function might be particularly useful when it is painful to explicitly type `np.array` for large-size codes.

In [12]:
num_qubit = 19
num_stabilizer = 9
num_logical_qubits = 1
h_indices = {
    0: [0,1,2,3],
    1: [1,3,5,6],
    2: [2,3,4,5,7,8],
    3: [4,7,10,11],
    4: [5,6,8,9,12,13],
    5: [10,11,14,15],
    6: [7,8,11,12,15,16],
    7: [12,13,16,17],
    8: [9,13,17,18],
}
a_indices = {
    0: [14,15,16,17,18],
}
logical_S_dict = {
    0: -1,
}
H, A, S = mlvtrans.convert_index_to_matrix(num_qubit=num_qubit, num_logical_qubits=num_logical_qubits, h_indices=h_indices, a_indices=a_indices, logical_S_dict=logical_S_dict)

In [13]:
mlvtrans.run(H_matrix=H, A_matrix=A, desired_logical_S=S)

Parity-check matrix H:
[[1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 0 1 1 0 0 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1]]

Coset representatives matrix A:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1]]

Desired logical phase-type gates:
 0 
 S̅†
Indices with S̅†: 0

Initial symplectic basis from Algorithm 1:
Pair 1:
  Lx = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1]   support = (14, 15, 16, 17, 18)   weight = 5
  Lz = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1]   support = (14, 15, 16, 17, 18)   weight = 5

Compatible symplectic basis:
Pair 1:
  Lx = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1]   support = (14, 15, 16, 17, 18)   weight = 5
  Lz = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1]   support = (14, 15, 16, 17, 18)   weight = 5

Physi

## 3. &nbsp; [[4,2,2]] code

In the [[4,2,2]] code, none of the coset representatives has odd weight, so a compatible symplectic basis does not exist.

In [14]:
H = np.array([
    [1, 1, 1, 1],
], dtype=int)
A = np.array([
    [0, 0, 1, 1],
    [1, 0, 1, 0],
], dtype=int)
S = (-1, 1)

In [15]:
mlvtrans.run(H_matrix=H, A_matrix=A, desired_logical_S=S)

Parity-check matrix H:
[[1 1 1 1]]

Coset representatives matrix A:
[[0 0 1 1]
 [1 0 1 0]]

Desired logical phase-type gates:
 0  1 
 S̅† S̅ 
Indices with S̅†: 0

Compatible symplectic basis does not exist: no vector in A has odd weight.
